In [1]:
# Core 
import datetime
import os
import glob
from IPython.display import HTML, Image, display
import tempfile
import shutil
import sys
import pickle

# Analysis 
import xarray as xr
import numpy as np
import pandas as pd
import pyproj as pp
import scipy as sp
import metpy as mp
import networkx 
from networkx.algorithms.components.connected import connected_components

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb, traceback
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint import animate
from tint.visualization import embed_mp4_as_gif


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

filenames = sorted(glob.glob(base + '2016/201612*/*.nc'))

In [6]:
# def test():
# Store filenames for an example day of data
base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

filenames = sorted(glob.glob(base + '2016/201612*/*.nc') + glob.glob(base + '2017/201701*/*.nc'))
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
         for fn in filenames)

# Define settings for tracking
settings = {
    'MIN_SIZE' : [15, 15, 15, 15], # This probably size in pixel units or km. 
    'FIELD_THRESH' : [45, 40, 35, 25],
    'ISO_THRESH' : [4, 4, 4, 4],
    'GS_ALT' : 3000,
    'LEVELS' : np.array(
        [[0, 2000], 
         [2000, 4000],
         [4000, 6000],
         [6000, 8000]]
    ),
    'TRACK_INTERVAL' : 1,
}

# Calculate high and low level tracks
tracks_obj  = tint.Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS', 
                  'TRACK_INTERVAL', 'ISO_THRESH'
                 ]:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
tracks_obj.get_tracks(grids)
    
# if __name__ == '__main__':
#     try:
#         test()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

No cells found in scan 8548       

time elapsed 24.7 minutes


In [7]:
with open('/g/data/w40/esh563/CPOL_analysis/tracks_obj_decjan1213.pkl', 'wb') as f:
    pickle.dump(tracks_obj, f)

In [3]:
with open('/g/data/w40/esh563/CPOL_analysis/tracks_obj.pkl', 'rb') as f:
    tracks_obj = pickle.load(f)

In [3]:
# Let's find the cells that were tracked for the most frames
tracks_obj.system_tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]
# tracks_obj.system_tracks['n_cores'][tracks_obj.system_tracks['n_cores'] == 5]

uid
6    6
8    3
4    3
7    2
5    2
3    2
9    1
2    1
1    1
0    1
dtype: int64

In [4]:
tracks_obj.system_tracks.xs('6', level='uid', drop_level=False)

,,,grid_x,grid_y,lon,lat,u,v,n_cores,x_vert_disp,y_vert_disp,tilt_mag,vel_dir,tilt_dir,sys_rel_tilt_dir
scan,time,uid,,,,,,,,,,,,,
2,2016-12-20 23:00:02,6,29.113,61.437,130.6670,-12.3296,NaN,NaN,1,2.014,-1.136,2.312292,NaN,-29.425243,NaN
3,2016-12-20 23:10:01,6,40.154,52.232,130.7682,-12.4106,18.401667,-15.341667,2,0.355,-1.318,1.364972,-39.818329,-74.925276,-35.106947
4,2016-12-20 23:20:02,6,47.314,49.909,130.8326,-12.4287,11.933333,-3.871667,1,0.121,-2.231,2.234279,-17.975206,-86.895561,-68.920355
5,2016-12-20 23:30:06,6,50.304,53.791,130.8603,-12.3927,4.983333,6.470000,1,2.493,-5.753,6.269933,52.395761,-66.571028,-118.966789
6,2016-12-20 23:40:02,6,57.908,50.051,130.9339,-12.4287,12.673333,-6.233333,1,-0.318,-4.381,4.392526,-26.190100,-94.151600,-67.961500
7,2016-12-20 23:50:02,6,66.019,46.367,131.0076,-12.4647,13.518333,-6.140000,2,2.734,-7.760,8.227536,-24.427454,-70.591641,-46.164187


In [15]:
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

filenames = sorted(glob.glob(base + '2016/20161220/*.nc') +
                   glob.glob(base + '2016/20161221/*.nc'))
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
         for fn in filenames)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")    

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_anim'

animate(tracks_obj, grids, out_path,
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True, keep_frames=True, dpi=100)



Animating 271 frames
Creating GIF - may take a few minutes.


In [4]:
def test():
    # Now we can view these cells from a lagrangian perspective
    grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator

    # Create directory for figures
    dt=str(datetime.datetime.now())[0:-7]
    dt=dt.replace(" ", "_")
    dt=dt.replace(":", "_")
    dt=dt.replace("-", "")  

    base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
    out_path = base_path + dt + '_tint_CPOL_lagrangian'
    animate(tracks_obj, grids, out_path, style='lagrangian', uid='112', alt=3000,
            keep_frames=True, dpi=100)

if __name__ == '__main__':
    try:
        test()
    except:
        extype, value, tb = sys.exc_info()
        traceback.print_exc()
        pdb.post_mortem(tb)

Animating 21 frames
Creating GIF - may take a few minutes.
